In [1]:
# import necessary packages to search for targets with continuous observation
import numpy as np
import pandas as pd
import lightkurve as lk  # main package used here
from astropy.coordinates import Angle

In [2]:
# this function is designed to search targets from the data set
# with given target WISE ID and returns available sectors of data
def search_target(target_name):
    # subtract the substring with char index
    RA_Hour = target_name[1:3]
    RA_Min = target_name[3:5]
    RA_Sec = target_name[5:10]
    RA_String = RA_Hour+'h'+RA_Min+'m'+RA_Sec+'s'
    # obtain the hour angle with astroPy function Angle()
    RA = str(np.around(Angle(RA_String).degree, 7))

    # similar to the procedures in RA
    DEC_Degree = target_name[10:13]
    DEC_Min = target_name[13:15]
    DEC_Sec = target_name[15:19]
    DEC_String = DEC_Degree+'d'+DEC_Min+'m'+DEC_Sec+'s'
    DEC = np.around(Angle(DEC_String).degree, 7)
    if DEC > 0:
        DEC = '+' + str(DEC)
    else:
        DEC = str(DEC)
    targetString = RA + DEC
    pfs = lk.search_tesscut(targetString)
    return pfs

In [3]:
# this function takes target pixel files and check whether the 
# corresponding object has continuous observation across sectors
def check_continuity(sec_num, pixel_file):
    # initialize the continuity flag as false
    continuous = False
    # traverse all missions
    for i in range(len(pixel_file.mission)-sec_num):
        # extract the sector number from the mission string
        curr_sec = int(pixel_file.mission[i].split()[-1])
        next_sec = int(pixel_file.mission[i+sec_num].split()[-1])
        # if the next sector is numerically adjacent to the current
        # sector, break the for loop and return true
        if next_sec-curr_sec == sec_num:
            continuous = True
            break
        else:
            continue
    return continuous

In [7]:
# this function traverses all targets in a given object type
# and return a list of targets with continuous observation
def search_continuity(ID, type):
    list = []

    for i in range(len(ID)):
        target_name = ID[i]
        pfs = search_target(target_name)
        if len(pfs) > 1:
            continuous = check_continuity(sec_num=2, pixel_file=pfs)
            if continuous:
                list.append([target_name, type])

    return list

In [8]:
# this function returns pandas dataframe from the TESS spreadsheet
def read_tess(sheetName):
    # open the Excel sheet that restores the tess data
    TESS = pd.ExcelFile('TESSRechecks_ALLDATA.xlsx')
    # select specified sheets
    sheet = pd.read_excel(TESS, sheetName)
    # extract only the list of WISE ID from the sheet
    return sheet['WISEID']

In [9]:
DD = read_tess('DebrisDisk').dropna()
EDD = read_tess('ExtremeDebrisDisk').dropna()
LateType = read_tess('LateTypeDisk').dropna()
Herbig = read_tess('HerbigAeBe+CBe').dropna()
YSO = read_tess('YSOE+YSOM').dropna()

In [10]:
DD_list = search_continuity(DD, "Debris Disk")
EDD_list = search_continuity(EDD, "Extreme Debris Disk")
LateType_list = search_continuity(LateType, "Late Type Disk")
Herbig_list = search_continuity(Herbig, "HerbigAeBe")
YSO_list = search_continuity(YSO, "YSO")

No data found for target "244.5355833-22.7051944".
No data found for target "256.9050417-6.4353333".
No data found for target "257.6829583-22.0046389".
No data found for target "275.3657917-2.1429167".
No data found for target "255.91375+13.60525".
No data found for target "224.387-20.1315833".
No data found for target "250.6474583-11.1746944".
No data found for target "251.4518333-26.6494722".
No data found for target "292.509-15.9756111".
No data found for target "245.5418333-21.6548611".
No data found for target "261.217625-18.8593056".
No data found for target "237.2115417-23.6859167".
No data found for target "307.92625-15.0011667".
No data found for target "258.653375-4.4844444".
No data found for target "244.76775-26.9615".
No data found for target "224.7656667-15.4352222".
No data found for target "264.8472083+1.9029722".
No data found for target "243.021-20.7279722".
No data found for target "277.4805417+0.8275".
No data found for target "253.6129583-2.4501111".
No data found 

In [16]:
# first combine all the list of targets with continuous observation
combined_list = [*DD_list, *EDD_list, *LateType_list, *Herbig_list, *YSO_list]
# open the .text file
with open(r'continuous_object.txt', 'w') as tf:
    # convert each row of the combined list as an item
    for object in combined_list:
        # write each item on a new line
        tf.write("%s\n" % object)
    print('Done')

Done
